# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [ ]:
agent = Agent(action_size)
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 3

### Main Training Loop

In [ ]:
vis_env_idx = 0
vis_env = envs[vis_env_idx]
e = 0
frame = 0
max_eval = -np.inf
reset_count = 0

while (e < EPISODES):
    step = 0
    assert(num_envs * env_mem_size == train_frame)
    frame_next_vals = []
    for i in range(num_envs):
        env = envs[i]
        #history = env.history
        #life = env.life
        #state, reward, done, info = [env.state, env.reward, env.done, env.info]
        for j in range(env_mem_size):
            step += 1
            frame += 1
            
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            
            if (i == vis_env_idx):
                vis_env._env.render()
            
            frame_next_state = get_frame(next_state)
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            
            env.life = env.info['ale.lives']
            r = env.reward
            
            agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
            if (j == env_mem_size-1):
                _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                frame_next_vals.append(frame_next_val)
            env.score += r
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            
            if (env.done):
                if (e % 50 == 0):
                    print('now time : ', datetime.now())
                    rewards.append(np.mean(evaluation_reward))
                    episodes.append(e)
                    pylab.plot(episodes, rewards, 'b')
                    pylab.savefig("./save_graph/spaceinvaders_ppo.png")
                    torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                    
                    if np.mean(evaluation_reward) > max_eval:
                        torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
                        max_eval = float(np.mean(evaluation_reward))
                        reset_count = 0
                    elif e > 5000:
                        reset_count += 1
                        if (reset_count == reset_max):
                            print("Training went nowhere, starting again at best model")
                            agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                            agent.update_target_net()
                            reset_count = 0
                    
                e += 1
                evaluation_reward.append(env.score)
                print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                  " evaluation reward:", np.mean(evaluation_reward))
                
                env.done = False
                env.score = 0
                env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                env.state = env.reset()
                env.life = number_lives
                get_init_state(env.history, env.state)
                
                
                
    agent.train_policy_net(frame, frame_next_vals)
    agent.update_target_net()

'''
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        curr_state = history[3,:,:]
        action, value = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        #r = np.clip(reward, -1, 1)
        r = reward
        """
        if terminal_state:
            r -= 20
        """
        # Store the transition in memory 
        
        agent.memory.push(deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
        # Start training after random sample generation
        if(frame % train_frame == 0):
            _, frame_next_val = agent.get_action(np.float32(history[1:, :, :]) / 255.)
            agent.train_policy_net(frame, frame_next_val)
            # Update the target network
            agent.update_target_net()
        score += r
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/spaceinvaders_ppo.png")
            torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 700 and len(evaluation_reward) > 40:
                torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                sys.exit()
'''

/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Training network. lr: 0.000250. clip: 0.100000


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:255: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:256: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:257: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Iteration 1: Policy loss: -1.446995. Value loss: 7.394697. Entropy: 1.790212.
Iteration 2: Policy loss: -1.446225. Value loss: 6.261855. Entropy: 1.785242.
Iteration 3: Policy loss: -1.420722. Value loss: 5.935813. Entropy: 1.780717.
Training network. lr: 0.000250. clip: 0.100000
Iteration 4: Policy loss: -5.571462. Value loss: 66.334053. Entropy: 1.766625.
Iteration 5: Policy loss: -5.743255. Value loss: 58.227165. Entropy: 1.776056.
Iteration 6: Policy loss: -5.517460. Value loss: 54.872612. Entropy: 1.779818.
Training network. lr: 0.000250. clip: 0.100000
Iteration 7: Policy loss: -2.662891. Value loss: 43.241081. Entropy: 1.764398.
Iteration 8: Policy loss: -2.797832. Value loss: 45.214840. Entropy: 1.773946.
Iteration 9: Policy loss: -2.675509. Value loss: 39.119541. Entropy: 1.762149.
now time :  2019-02-25 18:39:54.087786
episode: 1   score: 160.0  epsilon: 1.0    steps: 596  evaluation reward: 160.0


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

Training network. lr: 0.000134. clip: 0.053773
Iteration 15061: Policy loss: 0.913162. Value loss: 40.959023. Entropy: 0.406971.
Iteration 15062: Policy loss: 0.630818. Value loss: 22.996719. Entropy: 0.432329.
Iteration 15063: Policy loss: 0.807782. Value loss: 19.405457. Entropy: 0.422063.
episode: 6206   score: 240.0  epsilon: 1.0    steps: 477  evaluation reward: 292.25
episode: 6207   score: 260.0  epsilon: 1.0    steps: 847  evaluation reward: 291.45
episode: 6208   score: 420.0  epsilon: 1.0    steps: 966  evaluation reward: 293.15
Training network. lr: 0.000134. clip: 0.053773
Iteration 15064: Policy loss: 1.020694. Value loss: 27.772072. Entropy: 0.409107.
Iteration 15065: Policy loss: 1.049446. Value loss: 16.633759. Entropy: 0.382153.
Iteration 15066: Policy loss: 0.912709. Value loss: 14.768144. Entropy: 0.393872.
episode: 6209   score: 585.0  epsilon: 1.0    steps: 354  evaluation reward: 296.4
Training network. lr: 0.000134. clip: 0.053773
Iteration 15067: Policy loss: -0

Iteration 15126: Policy loss: -1.359000. Value loss: 9.809820. Entropy: 0.451981.
episode: 6228   score: 250.0  epsilon: 1.0    steps: 256  evaluation reward: 298.4
episode: 6229   score: 260.0  epsilon: 1.0    steps: 464  evaluation reward: 298.9
episode: 6230   score: 260.0  epsilon: 1.0    steps: 613  evaluation reward: 296.15
episode: 6231   score: 210.0  epsilon: 1.0    steps: 684  evaluation reward: 296.7
Training network. lr: 0.000134. clip: 0.053616
Iteration 15127: Policy loss: 3.427373. Value loss: 27.316761. Entropy: 0.421318.
Iteration 15128: Policy loss: 3.004091. Value loss: 13.481393. Entropy: 0.397344.
Iteration 15129: Policy loss: 3.394981. Value loss: 11.996657. Entropy: 0.404032.
Training network. lr: 0.000134. clip: 0.053616
Iteration 15130: Policy loss: -0.401854. Value loss: 20.575052. Entropy: 0.477096.
Iteration 15131: Policy loss: -0.176020. Value loss: 11.365851. Entropy: 0.488136.
Iteration 15132: Policy loss: -0.329001. Value loss: 11.621424. Entropy: 0.4756

Iteration 15187: Policy loss: 0.530889. Value loss: 37.311756. Entropy: 0.535032.
Iteration 15188: Policy loss: 0.398243. Value loss: 22.982355. Entropy: 0.554684.
Iteration 15189: Policy loss: 0.425622. Value loss: 17.369209. Entropy: 0.522993.
episode: 6254   score: 210.0  epsilon: 1.0    steps: 388  evaluation reward: 300.85
Training network. lr: 0.000134. clip: 0.053468
Iteration 15190: Policy loss: -0.593709. Value loss: 23.762758. Entropy: 0.448690.
Iteration 15191: Policy loss: -0.946374. Value loss: 13.661889. Entropy: 0.467052.
Iteration 15192: Policy loss: -1.036475. Value loss: 13.373862. Entropy: 0.455904.
episode: 6255   score: 220.0  epsilon: 1.0    steps: 881  evaluation reward: 300.95
Training network. lr: 0.000134. clip: 0.053468
Iteration 15193: Policy loss: 1.384543. Value loss: 22.057804. Entropy: 0.387665.
Iteration 15194: Policy loss: 1.279642. Value loss: 12.941618. Entropy: 0.388184.
Iteration 15195: Policy loss: 1.357941. Value loss: 11.541144. Entropy: 0.39969

Iteration 15253: Policy loss: 1.229157. Value loss: 36.098679. Entropy: 0.550969.
Iteration 15254: Policy loss: 1.247334. Value loss: 22.304287. Entropy: 0.543376.
Iteration 15255: Policy loss: 1.207406. Value loss: 16.578377. Entropy: 0.534481.
episode: 6276   score: 475.0  epsilon: 1.0    steps: 48  evaluation reward: 310.55
episode: 6277   score: 180.0  epsilon: 1.0    steps: 282  evaluation reward: 309.75
episode: 6278   score: 240.0  epsilon: 1.0    steps: 425  evaluation reward: 310.05
Training network. lr: 0.000133. clip: 0.053155
Iteration 15256: Policy loss: -0.177517. Value loss: 21.327196. Entropy: 0.401317.
Iteration 15257: Policy loss: -0.199387. Value loss: 18.309290. Entropy: 0.385056.
Iteration 15258: Policy loss: -0.229306. Value loss: 13.208226. Entropy: 0.423549.
episode: 6279   score: 210.0  epsilon: 1.0    steps: 666  evaluation reward: 310.35
Training network. lr: 0.000133. clip: 0.053155
Iteration 15259: Policy loss: -0.379808. Value loss: 47.974743. Entropy: 0.3

Iteration 15316: Policy loss: 0.531619. Value loss: 36.574646. Entropy: 0.246437.
Iteration 15317: Policy loss: 0.601416. Value loss: 20.919786. Entropy: 0.250453.
Iteration 15318: Policy loss: 0.838251. Value loss: 19.872307. Entropy: 0.258465.
Training network. lr: 0.000133. clip: 0.053008
Iteration 15319: Policy loss: 1.472726. Value loss: 39.756203. Entropy: 0.468328.
Iteration 15320: Policy loss: 1.337312. Value loss: 22.448505. Entropy: 0.484312.
Iteration 15321: Policy loss: 1.337524. Value loss: 17.362906. Entropy: 0.483633.
episode: 6300   score: 270.0  epsilon: 1.0    steps: 429  evaluation reward: 322.5
Training network. lr: 0.000133. clip: 0.053008
Iteration 15322: Policy loss: -0.427304. Value loss: 40.990711. Entropy: 0.525369.
Iteration 15323: Policy loss: -0.490727. Value loss: 24.340193. Entropy: 0.577584.
Iteration 15324: Policy loss: -0.596509. Value loss: 21.369080. Entropy: 0.537916.
now time :  2019-02-25 23:26:14.860272
episode: 6301   score: 415.0  epsilon: 1.0 

In [ ]:
torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")